In [1]:
import datacleaning
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance
import networkx as nx
import itertools
import matplotlib
import numpy as np
import scipy
# https://realpython.com/nltk-nlp-python/


In [2]:
data = datacleaning.clean_recipedata("recipes_raw_nosource_ar.json", n=5000)
print(np.shape(data))

data
# data.to_csv("data_small.csv")



(5000, 8)


,title,ingredients,instructions,picture_link,clean_ingredients,clean_instructions,clean_instructions_masked,title_words
5eA5nRW8VgbOry0hsA.SVnrGkt2AdzO,Southern-Style Chocolate Pound Cake,1 cup butter ADVERTISEMENT 1/2 cup shortening ...,Preheat oven to 350 degrees F (175 degrees C)....,v.IiJhm4GZSZAGtMDWpTfAe6vspLCiu,butter shortening white sugar eggs vanilla ext...,preheat oven degrees degrees grease flour loaf...,preheat oven degrees degrees grease loaf pans ...,"[southernstyle, chocolate, pound, cake]"
DIzfv.NycIqtwo58y7fteog1dKRw12O,Pumpkin Pie I,1 egg ADVERTISEMENT 1 tablespoon all-purpose f...,Preheat oven to 450 degrees F (230 degrees C)....,EbVR3lftwDSDeE1MRSGdi1evygKF/D6,egg allpurpose flour white sugar salt pumpkin ...,preheat oven degrees degrees add sugar gradual...,preheat oven degrees degrees add gradually bea...,"[pumpkin, pie, i]"
TFxp0RPK/1PxAnjYiZUS0KztkgYb3iW,Cinnamon Oatmeal Zucchini Cookies,"1 1/2 cups butter, softened ADVERTISEMENT 1 1/...",Preheat oven to 350 degrees F (175 degrees C)....,jG7CnWbsc8cYsoHbreIorq8Uvyzzk/6,butter softened white sugar eggs vanilla extra...,preheat oven degrees degrees grease baking she...,preheat oven degrees degrees grease sheets bea...,"[cinnamon, oatmeal, zucchini, cookies]"
3yH5Jc6HLmEnJw5ggLqI11oeBiSpwBy,Plum-Oat Drop Biscuits,2 tablespoons white sugar ADVERTISEMENT 1/2 te...,Preheat oven to 450 degrees F (230 degrees C)....,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,white sugar ground cinnamon allpurpose flour r...,preheat oven degrees degrees grease baking she...,preheat oven degrees degrees grease sheet comb...,"[plumoat, drop, biscuits]"
8lZak.EVdLP9/dukyN72DYHyjseFwV2,Ghirardelli Milk Chocolate Chip Cookies,2 1/4 cups all-purpose flour ADVERTISEMENT 1 t...,Preheat the oven to 375 degrees F. Stir togeth...,50UyVqBQayirbQ47M.8oJvkETheUGFW,allpurpose flour baking soda salt unsalted but...,preheat oven degrees stir together flour bakin...,preheat oven degrees stir together set aside b...,"[ghirardelli, milk, chocolate, chip, cookies]"
...,...,...,...,...,...,...,...,...
z5ZKH66nGOR2cgkzDl5vGj4DPX/J95O,Sun-Dried Tomato With Fresh Basil Spread,1/2 cup oil-packed sun-dried tomatoes ADVERTIS...,Place sun-dried tomatoes in the work bowl of a...,IV7I3CRDxEMTrdHzOCE3YHxewYs0JQW,oilpacked sundried tomatoes basil leaves ounce...,place sundried tomatoes work bowl food process...,place work bowl food processor pulse coarsely ...,"[sundried, tomato, with, fresh, basil, spread]"
g1mQzPtIoBSaOaaHSkMj7cObWt0r5.K,Banana Split Martini,ice as needed ADVERTISEMENT 2 fluid ounces whi...,Fill a cocktail shaker with ice; add white cho...,qE58a7Z1Au0GXvPO188iHZZVqna9hLa,ice needed fluid ounces white chocolate liqueu...,fill cocktail shaker ice add white chocolate l...,fill cocktail shaker add creme de cover shake ...,"[banana, split, martini]"
dsDOddsQtDp7xomWY2mQwcoTyLeUfYa,Citrus Glazed Banana Squash,"1 1/2 pounds banana squash, peeled and cubed A...",Melt butter in a large skillet over medium hea...,SkAAXlzdZZcQ6UmAO72KI82FiqpAG9y,banana squash peeled cubed butter water needed...,melt butter large skillet medium heat add squa...,melt large skillet medium heat add cubes cover...,"[citrus, glazed, banana, squash]"
CdxL1ghjiOgZTF8mPLi/JdaUW74ARmq,Classic Spanish Sangria,1 lemon ADVERTISEMENT 1 lime ADVERTISEMENT 1 o...,"Have the fruit, rum, wine, and orange juice we...",xCSyOeooKYofbXHuZpV5h7prJrWUDhK,lemon lime orange rum white sugar bottle dry r...,fruit rum wine orange juice well chilled slice...,fruit well chilled slice thin rounds place lar...,"[classic, spanish, sangria]"


In [3]:
data["ingredient_words"] = data.clean_ingredients.apply(word_tokenize)
data["instruction_words"] = data.clean_instructions_masked.apply(word_tokenize)

data.to_csv("data_small.csv")

In [ ]:
# ingredient_units = {'inch', 'ml', 'milliliter','milliliters','liters','teaspoons', 'l','liter','teaspoon','t','tsp','tablespoon','tablespoons','tbl','tbs','tbsp','ounce','oz','fl','cup','cups','c','pint','pints','pt','p','quart','quarts','qt','gal','gals','gallon','gallons','g','mg','milligram','milligrams','gram','grams','pound','pounds','lb','lbs','c','f'}

In [ ]:
# stop_words = set(stopwords.words("english"))
# stop_words = stop_words.union(ingredient_units)

# def filter_stop_words(words):
#     output = list()
#     for word in words:
#         if word.casefold() not in stop_words:
#             output.append(word)
#     return(output)

# data["instruction_words"] = data["instruction_words"].apply(filter_stop_words)
# data["ingredient_words"] = data["ingredient_words"].apply(filter_stop_words)
data['index'] = data.index
data.to_csv("data_small.csv")


In [ ]:
data["ingredient_words"]

In [ ]:
all_instruction_words = list(itertools.chain.from_iterable(data["instruction_words"]))
all_ingredient_words = list(itertools.chain.from_iterable(data["ingredient_words"]))

In [ ]:
len(FreqDist(all_ingredient_words).items())

In [ ]:
# Instructions
common_instruction_words = []

for item in FreqDist(all_instruction_words).items():
    if (item[1]) > (len(data)*.01): # remove words that appear less than once per hundred recipes
        common_instruction_words.append(item)


common_instruction_words = sorted(common_instruction_words, key=lambda x: x[1], reverse=True)

common_instruction_words_dict = {}
for i in range(len(common_instruction_words)):
    common_instruction_words_dict[common_instruction_words[i][0]] = i



# Ingredients
common_ingredient_words = []
for item in FreqDist(all_ingredient_words).items():
    if (item[1]) > (len(data)*.01): # remove words that appear less than once per hundred recipes
        common_ingredient_words.append(item)

common_ingredient_words = sorted(common_ingredient_words, key=lambda x: x[1], reverse=True)

common_ingredient_words_dict = {}
for i in range(len(common_ingredient_words)):
    common_ingredient_words_dict[common_ingredient_words[i][0]] = i

In [ ]:
len(data)*.01

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
ingred = ((pd.DataFrame({a:[b] for a, b in common_ingredient_words})).T.reset_index())
ingred = ingred[ingred[0] > 1500]

inst = ((pd.DataFrame({a:[b] for a, b in common_instruction_words})).T.reset_index())
inst = inst[inst[0] > 2500]

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(6, 3))

a = sns.barplot(y=ingred[0], x=ingred['index'], ax = axes[0])
axes[0].tick_params(axis='x', rotation=90)
a.set_xlabel("")
a.set_ylabel("Count")
a.set_title("Ingredients")

b = sns.barplot(y=inst[0], x=inst['index'], ax = axes[1])
axes[1].tick_params(axis='x', rotation=90)
b.set_xlabel("")
b.set_ylabel("")
b.set_title("Instructions")

fig.suptitle("Word Frequencies")
fig.tight_layout()

plt.savefig('figs/original_wordcounts.pdf')

In [ ]:
common_instruction_words_dict

In [ ]:
instruction_vectorizer = CountVectorizer(vocabulary=common_instruction_words_dict)

ingredient_vectorizer = CountVectorizer(vocabulary=common_ingredient_words_dict)

In [ ]:
A = instruction_vectorizer.fit_transform(data["instructions"])
A_ingredients = ingredient_vectorizer.fit_transform(data["ingredients"])

In [ ]:
# B = cosine_similarity(A, A[1,:])

In [ ]:
# np.histogram(B, bins=4)

In [ ]:
# A2 = A[0:100,:]
# print(np.shape(A2)[0])
# G2 = nx.Graph()
# for i, attr in data[0:100].iterrows():
#     G2.add_node(i, title = attr[0], ingredients = attr[1], instructions = attr[2])

# for i in range((np.shape(A2))[0]):
#     # if i%1000 == 0: 
#     #     print(i/(np.shape(A2))[0])
#     current_node = data.index[i]
#     current_node_similarity = cosine_similarity(A2, A2[i,:])
#     for j in range((np.shape(A2))[0]):
#         target_node = data.index[j]
#         similarity = current_node_similarity[j]
#         if (similarity > 0.5 and current_node != target_node): #arbitrary cutoff
#             G2.add_edge(current_node, target_node, weight = float(similarity))

In [ ]:
# A2 = A_ingredients[0:100,:]
# G2 = nx.Graph()
# for i, attr in data[0:100].iterrows():
#     G2.add_node(i, title = attr[0])

# for i in range((np.shape(A2))[0]):
#     if i%1000 == 0: 
#         print(i/(np.shape(A2))[0])
#     current_node = data.index[i]
#     current_node_similarity = cosine_similarity(A2, A2[i,:])
#     edges_to_add = np.argwhere(current_node_similarity > .5)[:,0] #arbitrary cutoff
#     for j in edges_to_add:
#         target_node = data.index[j]
#         similarity = current_node_similarity[j]
#         if (current_node != target_node): 
#             G2.add_edge(current_node, target_node, weight = float(similarity))

In [ ]:
# current_node_similarity = cosine_similarity(A, A[1,:])



In [ ]:
# thing = np.vstack((np.ravel(current_node_similarity),np.array(range(39522))))

# np.shape(thing[:,] > .5)

In [ ]:
G = nx.Graph()
for i, attr in data.iterrows():
    G.add_node(i, title = attr[0])

for i in range((np.shape(A))[0]):
    if i%1000 == 0: 
        print(i/(np.shape(A))[0])
    current_node = data.index[i]
    current_node_similarity = cosine_similarity(A, A[i,:])
    edges_to_add = np.argwhere(current_node_similarity > .5)[:,0] #arbitrary cutoff
    for j in edges_to_add:
        target_node = data.index[j]
        similarity = current_node_similarity[j]
        if (current_node != target_node): 
            G.add_edge(current_node, target_node, weight = float(similarity))

In [ ]:
nx.write_gexf(G, "recipe_instruction_small.gexf")

In [ ]:
G_ingr = nx.Graph()
for i, attr in data.iterrows():
    G_ingr.add_node(i, title = attr[0])

for i in range((np.shape(A_ingredients))[0]):
    if i%1000 == 0: 
        print(i/(np.shape(A_ingredients))[0])
    current_node = data.index[i]
    current_node_similarity = cosine_similarity(A_ingredients, A_ingredients[i,:])
    edges_to_add = np.argwhere(current_node_similarity > .5)[:,0] #arbitrary cutoff
    for j in edges_to_add:
        target_node = data.index[j]
        similarity = current_node_similarity[j]
        if (current_node != target_node): 
            G_ingr.add_edge(current_node, target_node, weight = float(similarity))

In [ ]:
nx.write_gexf(G_ingr, "recipe_ingredient_small.gexf")

In [ ]:
data.iloc[1].iloc[2]

In [ ]:
# Instructions
common_instruction_words = []

for item in FreqDist(all_instruction_words).items():
    # if (item[1]) > (len(data)*.01): # remove words that appear less than once per hundred recipes
    common_instruction_words.append(item)


common_instruction_words = sorted(common_instruction_words, key=lambda x: x[1], reverse=True)

common_instruction_words_dict = {}
for i in range(len(common_instruction_words)):
    common_instruction_words_dict[common_instruction_words[i][0]] = i



# Ingredients
common_ingredient_words = []
for item in FreqDist(all_ingredient_words).items():
    # if (item[1]) > (len(data)*.01): # remove words that appear less than once per hundred recipes
    common_ingredient_words.append(item)

common_ingredient_words = sorted(common_ingredient_words, key=lambda x: x[1], reverse=True)

common_ingredient_words_dict = {}
for i in range(len(common_ingredient_words)):
    common_ingredient_words_dict[common_ingredient_words[i][0]] = i


instruction_vectorizer = CountVectorizer(vocabulary=common_instruction_words_dict)

ingredient_vectorizer = CountVectorizer(vocabulary=common_ingredient_words_dict)

A = instruction_vectorizer.fit_transform(data["instructions"])
A_ingredients = ingredient_vectorizer.fit_transform(data["ingredients"])


G2 = nx.Graph()
for i, attr in data.iterrows():
    G2.add_node(i, title = attr[0])

for i in range((np.shape(A))[0]):
    if i%1000 == 0: 
        print(i/(np.shape(A))[0])
    current_node = data.index[i]
    current_node_similarity = cosine_similarity(A, A[i,:])
    edges_to_add = np.argwhere(current_node_similarity > .5)[:,0] #arbitrary cutoff
    for j in edges_to_add:
        target_node = data.index[j]
        similarity = current_node_similarity[j]
        if (current_node != target_node): 
            G2.add_edge(current_node, target_node, weight = float(similarity))



G2_ingr = nx.Graph()
for i, attr in data.iterrows():
    G2_ingr.add_node(i, title = attr[0])

for i in range((np.shape(A_ingredients))[0]):
    if i%1000 == 0: 
        print(i/(np.shape(A_ingredients))[0])
    current_node = data.index[i]
    current_node_similarity = cosine_similarity(A_ingredients, A_ingredients[i,:])
    edges_to_add = np.argwhere(current_node_similarity > .5)[:,0] #arbitrary cutoff
    for j in edges_to_add:
        target_node = data.index[j]
        similarity = current_node_similarity[j]
        if (current_node != target_node): 
            G2_ingr.add_edge(current_node, target_node, weight = float(similarity))

In [ ]:
nx.write_gexf(G2, "recipe_instruction_nofilter.gexf")
nx.write_gexf(G2_ingr, "recipe_ingredient_nofilter.gexf")

In [ ]:
scipy.sparse.linalg.norm(nx.adjacency_matrix(G_ingr) - nx.adjacency_matrix(G2_ingr))

In [ ]:
scipy.sparse.linalg.norm(nx.adjacency_matrix(G) - nx.adjacency_matrix(G2))

In [ ]:
scipy.sparse.linalg.norm(nx.adjacency_matrix(G) - nx.adjacency_matrix(G_ingr))

In [ ]:
scipy.sparse.linalg.norm(nx.adjacency_matrix(G2_ingr) - nx.adjacency_matrix(G2))

In [ ]:
data

In [ ]:
instructions = list(data["clean_instructions_masked"])
ingredients = list(data["clean_ingredients"])
tfidf = TfidfVectorizer()
tfidf2 = TfidfVectorizer()

In [ ]:
A = tfidf.fit_transform(instructions)
A_ingredients = tfidf2.fit_transform(ingredients)

In [ ]:
feature_array = np.array(tfidf.get_feature_names_out())
tfidf_sorting = np.argsort(tfidf.idf_)[::-1]



instructions_idf = pd.DataFrame({"word": tfidf.get_feature_names_out(), "weight": tfidf.idf_})
ingredients_idf = pd.DataFrame({"word": tfidf2.get_feature_names_out(), "weight": tfidf2.idf_})

instructions_idf.to_csv("instructions_idf.csv")
ingredients_idf.to_csv("ingredients_idf.csv")


# n = 100
# top_n = feature_array[tfidf_sorting][:n]
# top_n
ingredients_idf.sort_values(by="weight")

In [ ]:
feature_array = np.array(tfidf.get_feature_names_out())
tfidf_sorting = np.argsort(tfidf.idf_)[::-1]

n = 100
top_n = feature_array[tfidf_sorting][:n]
top_n

In [ ]:
tfidf.idf_

In [ ]:
G3 = nx.Graph()
for i, attr in data.iterrows():
    G3.add_node(i, title = attr[0])

for i in range((np.shape(A))[0]):
    if i%1000 == 0: 
        print(i/(np.shape(A))[0])
    current_node = data.index[i]
    current_node_similarity = cosine_similarity(A, A[i,:])
    edges_to_add = np.argwhere(current_node_similarity > .5)[:,0] #arbitrary cutoff
    for j in edges_to_add:
        target_node = data.index[j]
        similarity = current_node_similarity[j]
        if (current_node != target_node): 
            G3.add_edge(current_node, target_node, weight = float(similarity))



G3_ingr = nx.Graph()
for i, attr in data.iterrows():
    G3_ingr.add_node(i, title = attr[0])

for i in range((np.shape(A_ingredients))[0]):
    if i%1000 == 0: 
        print(i/(np.shape(A_ingredients))[0])
    current_node = data.index[i]
    current_node_similarity = cosine_similarity(A_ingredients, A_ingredients[i,:])
    edges_to_add = np.argwhere(current_node_similarity > .5)[:,0] #arbitrary cutoff
    for j in edges_to_add:
        target_node = data.index[j]
        similarity = current_node_similarity[j]
        if (current_node != target_node): 
            G3_ingr.add_edge(current_node, target_node, weight = float(similarity))

In [ ]:
nx.write_gexf(G3, "recipe_instruction_tfidf.gexf")
nx.write_gexf(G3_ingr, "recipe_ingredient_tfidf.gexf")

In [ ]:
G4 = nx.Graph()
for i, attr in data.iterrows():
    G4.add_node(i, title = attr[0])

for i in range((np.shape(A))[0]):
    if i%1000 == 0: 
        print(i/(np.shape(A))[0])
    current_node = data.index[i]
    current_node_similarity = cosine_similarity(A, A[i,:])
    edges_to_add = np.argsort(np.squeeze(current_node_similarity))[::-1]
    edges_to_add = np.squeeze(edges_to_add[:3])
    # b = np.argwhere(current_node_similarity > .5)[:,0] #arbitrary cutoff
    for j in edges_to_add:
        target_node = data.index[j]
        similarity = current_node_similarity[j]
        if (current_node != target_node): 
            G4.add_edge(current_node, target_node, weight = float(similarity))



G4_ingr = nx.Graph()
for i, attr in data.iterrows():
    G4_ingr.add_node(i, title = attr[0])

for i in range((np.shape(A_ingredients))[0]):
    if i%1000 == 0: 
        print(i/(np.shape(A_ingredients))[0])
    current_node = data.index[i]
    current_node_similarity = cosine_similarity(A_ingredients, A_ingredients[i,:])
    # edges_to_add = np.argwhere(current_node_similarity > .5)[:,0] #arbitrary cutoff
    edges_to_add = np.argsort(np.squeeze(current_node_similarity))[::-1]
    edges_to_add = np.squeeze(edges_to_add[:3])
    for j in edges_to_add:
        target_node = data.index[j]
        similarity = current_node_similarity[j]
        if (current_node != target_node): 
            G4_ingr.add_edge(current_node, target_node, weight = float(similarity))

In [ ]:
current_node_similarity

In [ ]:
nx.write_gexf(G4, "recipe_instruction_ranked_tfidf.gexf")
nx.write_gexf(G4_ingr, "recipe_ingredient_ranked_tfidf.gexf")

In [ ]:
data.loc["k9Wo2oKtwveioGZll5XyvYnviGvJr72"]

In [ ]:
data.loc["iPiuCa80W95Q1hwpAvNcQoAK3GOonMq"]

In [ ]:
instruction_vectorizer = CountVectorizer(max_df= .5, min_df= 0.0, binary=True)

ingredient_vectorizer = CountVectorizer(max_df= .5, min_df= 0.0, binary=True)

A = instruction_vectorizer.fit_transform(data["instructions"])
A_ingredients = ingredient_vectorizer.fit_transform(data["ingredients"])

In [ ]:
G5 = nx.Graph()
for i, attr in data.iterrows():
    G5.add_node(i, title = attr[0])

for i in range((np.shape(A))[0]):
    if i%1000 == 0: 
        print(i/(np.shape(A))[0])
    current_node = data.index[i]
    current_node_similarity = cosine_similarity(A, A[i,:])
    edges_to_add = np.argwhere(current_node_similarity > .5)[:,0] #arbitrary cutoff
    for j in edges_to_add:
        target_node = data.index[j]
        similarity = current_node_similarity[j]
        if (current_node != target_node): 
            G5.add_edge(current_node, target_node, weight = float(similarity))


G5_ingr = nx.Graph()
for i, attr in data.iterrows():
    G5_ingr.add_node(i, title = attr[0])

for i in range((np.shape(A_ingredients))[0]):
    if i%1000 == 0: 
        print(i/(np.shape(A_ingredients))[0])
    current_node = data.index[i]
    current_node_similarity = cosine_similarity(A_ingredients, A_ingredients[i,:])
    edges_to_add = np.argwhere(current_node_similarity > .5)[:,0] #arbitrary cutoff
    for j in edges_to_add:
        target_node = data.index[j]
        similarity = current_node_similarity[j]
        if (current_node != target_node): 
            G5_ingr.add_edge(current_node, target_node, weight = float(similarity)) 

In [ ]:
nx.write_gexf(G5, "recipe_instruction_binary.gexf")
nx.write_gexf(G5_ingr, "recipe_ingredient_binary.gexf")